In [1]:
!apt install swig cmake ffmpeg xvfb python3-opengl
!pip install stable-baselines3==2.0.0a5 gymnasium[box2d] huggingface_sb3 pyvirtualdisplay imageio[ffmpeg]

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
python3-opengl is already the newest version (3.1.5+dfsg-1).
swig is already the newest version (4.0.2-1ubuntu1).
cmake is already the newest version (3.22.1-1ubuntu1.22.04.2).
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
xvfb is already the newest version (2:21.1.4-2ubuntu1.7~22.04.8).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [ ]:
import os
os.kill(os.getpid(), 9)

In [1]:
from pyvirtualdisplay import Display
virtual_display = Display(visible=0, size=(1400, 900))
virtual_display.start()

In [2]:
import gymnasium as gym
env = gym.make("BipedalWalker-v3", hardcore=True)
env.reset()

(array([ 2.7461224e-03,  8.8099468e-06, -1.1487805e-03, -1.6000059e-02,
         9.2434227e-02,  2.6668247e-03,  8.5984993e-01, -7.1759778e-04,
         1.0000000e+00,  3.2764878e-02,  2.6667158e-03,  8.5356247e-01,
        -1.8145231e-03,  1.0000000e+00,  4.4081339e-01,  4.4581950e-01,
         4.6142212e-01,  4.8954949e-01,  5.3410202e-01,  6.0246021e-01,
         7.0914787e-01,  8.8593054e-01,  1.0000000e+00,  1.0000000e+00],
       dtype=float32),
 {})

In [3]:
print("_____OBSERVATION SPACE_____ \n")
print("Observation Space Shape", env.observation_space.shape)
print("Sample observation", env.observation_space.sample())

_____OBSERVATION SPACE_____ 

Observation Space Shape (24,)
Sample observation [ 2.0734482  -1.0584437  -4.4860005   3.7951388   0.7108759  -3.4183807
  1.9083704  -2.992434    2.733271    3.1004853   1.7032919  -0.55823857
 -0.60968304  0.68147886  0.08381164 -0.2775325   0.37168753 -0.56543285
  0.8524479   0.1416852   0.7077905   0.53145933  0.19349855 -0.5513372 ]


In [4]:
print("\n _____ACTION SPACE_____ \n")
print("Action Space Shape", env.action_space.shape)
print("Action Space Sample", env.action_space.sample())


 _____ACTION SPACE_____ 

Action Space Shape (4,)
Action Space Sample [ 0.49625555 -0.89484084 -0.06804753  0.7038753 ]


In [5]:
from stable_baselines3.common.env_util import make_vec_env
env = make_vec_env('BipedalWalker-v3', n_envs=16)

In [6]:
from stable_baselines3 import A2C
model = A2C(
    policy = 'MlpPolicy',
    env = env,
    verbose=1)

Using cpu device


In [7]:
from wasabi import Printer
import numpy as np
from stable_baselines3.common.base_class import BaseAlgorithm
from pathlib import Path
import tempfile
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import (
    DummyVecEnv,
    VecEnv,
    VecVideoRecorder,
)

/usr/local/lib/python3.10/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
msg = Printer()

In [9]:
def generate_replay(
    model: BaseAlgorithm,
    eval_env: VecEnv,
    video_length: int,
    is_deterministic: bool,
    local_path: Path,
):
    """
    Generate a replay video of the agent
    :param model: trained model
    :param eval_env: environment used to evaluate the agent
    :param video_length: length of the video (in timesteps)
    :param is_deterministic: use deterministic or stochastic actions
    :param local_path: path of the local repository
    """
    # This is another temporary directory for video outputs
    # SB3 created a -step-0-to-... meta files as well as other
    # artifacts which we don't want in the repo.
    with tempfile.TemporaryDirectory() as tmpdirname:
        # Step 1: Create the VecVideoRecorder
        env = VecVideoRecorder(
            eval_env,
            tmpdirname,
            record_video_trigger=lambda x: x == 0,
            video_length=video_length,
            name_prefix="",
        )

        obs = env.reset()
        lstm_states = None
        episode_starts = np.ones((env.num_envs,), dtype=bool)

        try:
            for _ in range(video_length):
                action, lstm_states = model.predict(
                    obs,
                    state=lstm_states,
                    episode_start=episode_starts,
                    deterministic=is_deterministic,
                )
                obs, _, episode_starts, _ = env.step(action)

            # Save the video
            env.close()

            # Convert the video with x264 codec
            inp = env.video_recorder.path
            out = local_path
            os.system(f"ffmpeg -y -i {inp} -vcodec h264 {out}".format(inp, out))
            print(f"Video saved to: {out}")
        except KeyboardInterrupt:
            pass
        except Exception as e:
            msg.fail(str(e))
            # Add a message for video
            msg.fail(
                "We are unable to generate a replay of your agent"
            )

In [15]:
model.learn(total_timesteps=580000)

# Save the model
model_name = "a2c-BipedalWalker-v3"
model.save(model_name)

------------------------------------
| rollout/              |          |
|    ep_len_mean        | 85.5     |
|    ep_rew_mean        | -112     |
| time/                 |          |
|    fps                | 2101     |
|    iterations         | 100      |
|    time_elapsed       | 3        |
|    total_timesteps    | 8000     |
| train/                |          |
|    entropy_loss       | -4.84    |
|    explained_variance | 0.59     |
|    learning_rate      | 0.0007   |
|    n_updates          | 4849     |
|    policy_loss        | 0.323    |
|    std                | 0.816    |
|    value_loss         | 1.43     |
------------------------------------
------------------------------------
| rollout/              |          |
|    ep_len_mean        | 147      |
|    ep_rew_mean        | -103     |
| time/                 |          |
|    fps                | 2164     |
|    iterations         | 200      |
|    time_elapsed       | 7        |
|    total_timesteps    | 16000    |
|

In [16]:
import os
video_dir = "."
video_name = "replay.mp4"
env_id = "BipedalWalker-v3"

In [17]:
generate_replay(
    model = model,
    eval_env=DummyVecEnv([lambda: Monitor(gym.make(env_id, render_mode="rgb_array"))]),
    video_length=1000,
    is_deterministic=True,
    local_path=os.path.join(video_dir, video_name)
)

Saving video to /tmp/tmp1l4dv87f/-step-0-to-step-1000.mp4
Moviepy - Building video /tmp/tmp1l4dv87f/-step-0-to-step-1000.mp4.
Moviepy - Writing video /tmp/tmp1l4dv87f/-step-0-to-step-1000.mp4



Moviepy - Done !
Moviepy - video ready /tmp/tmp1l4dv87f/-step-0-to-step-1000.mp4
Video saved to: ./replay.mp4


In [18]:
from IPython.display import HTML
from base64 import b64encode
mp4 = open('replay.mp4','rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
HTML("""
<video width=600 controls>
      <source src="%s" type="video/mp4">
</video>
""" % data_url)

In [19]:
from stable_baselines3.common.evaluation import evaluate_policy

eval_env = Monitor(gym.make("BipedalWalker-v3"))
mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=10, deterministic=True)
print(f"mean_reward={mean_reward:.2f} +/- {std_reward}")

mean_reward=262.39 +/- 2.741506200957525
